# SEN163A - Fundamentals of Data Analytics
# Assignment 1 - Data consistency
### Ir. Jacopo De Stefani - [J.deStefani@tudelft.nl](mailto:J.deStefani@tudelft.nl)
### Joao Pizani Flor, M.Sc. - [J.p.pizaniflor@tudelft.nl](mailto:J.p.pizaniflor@tudelft.nl)

## Group X
- Emmanuel M Boateng - '5617642'
- Philip Busscher - '4611993'
- Student 3 - ''Student ID3''
- Student 4 - ''Student ID4''

## Evaluation criteria
The final grade for this assignment will be calculated based on the following criteria:

- $\textbf{Quality of the report}$ - 35\%
    - Reasonable formatting of the document and used citation appropriately
    - Use of proper English (typos, grammar)
    - Code script deliverable
    - Code quality
    - Problem Description
    - Dataset Description
    - Limitations
    - Conclusion/Action recommendations

- $\textbf{Identification of the problems in the database}$ - 65\%


# Introduction 

### Example text cell with formula

Bullet list:

- Item 1
- Item 2

Numbered list:

1. Item 1
2. Item 2


#### Example of simple equation
\begin{equation}
e = mc^2
\end{equation}

#### Example of matrix equation - Cross product formula:

\begin{equation*}
\mathbf{V}_1 \times \mathbf{V}_2 =  \begin{vmatrix}
\mathbf{i} & \mathbf{j} & \mathbf{k} \\
\frac{\partial X}{\partial u} &  \frac{\partial Y}{\partial u} & 0 \\
\frac{\partial X}{\partial v} &  \frac{\partial Y}{\partial v} & 0
\end{vmatrix}
\end{equation*}

#### Example of multiline equation - The Lorenz Equations:

\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

#### Example of Markdown Table:

| This | is   |
|------|------|
|   a  | table|


# Dataset Description

In [ ]:
### Read and load the database

import pandas as pd
import matplotlib as plt
import sqlite3 as sq3
import numpy as np

dbfile = 'transaction_data.db' # path to the file 

conn = sq3.connect(dbfile)  # Create a SQL connection to our SQLite database
curs = conn.cursor()

##Check the tables in the database
table_lst = [a for a in curs.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()]
print(table_lst) # here is you table list

##get the table info 
for row in curs.execute('PRAGMA table_info(transaction_data)'):    
    print(row)

##read the SQL data and print the first five lines
df = pd.read_sql_query("SELECT * FROM transaction_data", conn) 
##df.head()

##create a list of all columns 
col_names = []
for row in curs.execute('PRAGMA table_info(transaction_data)'):
    names = row[1]
    col_names.append(names)



In [ ]:
df.head(10)

In [ ]:
## Get the data description

##convert data types of columns from string to numeric
convert_list =["amount", 
               "oldbalanceOrig", 
               "newbalanceOrig", 
               "oldbalanceDest", 
               "newbalanceDest"]

## convert the str columns of convert_list to numeric
df[convert_list] = df[convert_list].apply(pd.to_numeric)


data_shape = df.shape
print('\nDataFrame Shape :', data_shape)
print('\nNumber of data points :', data_shape[0])
print('\nNumber of data variables :', data_shape[1])


df_MinMax = df[['amount','timestamp', 'newbalanceOrig', 'newbalanceDest']].agg(['min','max'])
print (df_MinMax)

chnk_sz = 7
data_chunks = np.array_split(df, chnk_sz)

df_meanAmount = df['amount'].mean()
print('\nMean amount transferred :', df_meanAmount)

##some info about the dataset (already done with sqlite 
##df.info(verbose=True, buf=None, max_cols=True, memory_usage=None, show_counts=True)

## Plotting some data. Not so informative. TBD: Perhaps filter and plot based on amounts
df.plot(x='amount', y='timestamp', kind='scatter',
        figsize=(10,6),
        title='Amounts transferred over timestamps')

orig_n = len(pd.unique(df['nameOrig']))
dest_n = len(pd.unique(df['nameDest']))


print("\nNo.of. origin accounts :", orig_n)
print("\nNo.of. destination accounts :", dest_n)

##df.plot(y=['amount'])

##TBD count timestamp with most # of transactions


# Dataset Consistency

In [ ]:
### Your code goes here

##splitting data based on timestamps
grouped = df.groupby(df.timestamp)

##get a specific timestamp group 
timestamp = 15
ts_1 = grouped.get_group(timestamp)
ts_1.head()

######
#splitting data into chunks (for future assignments)
#import numpy as np
#chnk_sz = 3
#np.array_split(df, chnk_sz)
######

##Check for duplicates in each time stamp
ts_idx = [] 
for ts in range(1,len(grouped)):
    g = grouped.get_group(ts)
    g_len = len(g)
    unq_rows = g.drop_duplicates()
    new_g_len = len(unq_rows)
    if new_g_len != g_len:
        ts_idx.append(ts)
        print("g_len: ", g_len, "unique rows: ", new_g_len)

if len(ts_idx) > 0:
    print("Duplicates exist in timestamps", *ts_idx)
else:
   print("Data is without duplicates", *ts_idx)


# create a function out of this later
#for items in range(len(grouped.get_group(2))):
#    group_obj = grouped.get_group(2)
#    if group_obj.iloc[items,3] == group_obj.iloc[items,6] :
#        print("id",group_obj.iloc[items,3] "same orig: ",group_obj.iloc[items,3], "and dest: ",group_obj.iloc[items,6])





# Fraud Identification

In [ ]:
### Your code goes here

##create functions to loop through groups
##Consider tweaking the absolute tolerance(atol) argument 
def inconsitentNewBalance(group_obj):
    for index, row in group_obj.iterrows():
        if np.isclose(row['sumDest'], row['newbalanceDest'], atol=1e-02):
            row['FraudType'] = "Legitimate"  ## Modify from this line here to create some statistics
        else:
            row['FraudType'] = "invalidDestBalance"
            

def inconsitentOldBalance(group_obj):
    for index, row in group_obj.iterrows():
        if np.isclose(row['diffOrig'], row['oldbalanceOrig'], atol=1e-02):
            row['FraudType'] = "Legitimate" ## Modify from this line here to add some statistics
        else:
            row['FraudType'] = "invalidOrigBalance"
            
## small amount transfers between same accounts. 
#def laundering(group_obj):
#    for index, row in group_obj.iterrows(): 
#....

#sum up the columns 
df['sumDest'] = df.loc[: , ['amount', 'oldbalanceDest']].sum(axis=1)
df['diffOrig'] = df['oldbalanceOrig'] - df['amount']
#df.head()

grouped = df.groupby(df.timestamp)


##test fraudulent activity balance inconsitency
g = grouped.get_group(301)
for index, row in g.iterrows():
    if np.isclose(row['sumDest'], row['newbalanceDest'], atol=1e-02):
        row['FraudType'] = "Legitimate"  ## change from this line here to create some statistics
    else:
        row['FraudType'] = "invalidDestBalance"
        print(row)

##Loop through grouped data 
#for datapoints in grouped:
#    inconsitentOldBalance(datapoints)
#    inconsitentNewBalance(datapoints)

##group data by transfer amount (possibly add timestamp grouping as well)
transferSum = df.groupby(df.amount)
transferSum.groups
#for index, row in transferSum.iterrows():
#    ids = row["amount"]
#    print(ids)
    #row[ids.astype(int).isin(ids[ids.duplicated()])].sort("amount")
    
    
# print first 10 lines of the dataframe

df_first_10 = df.head(10)

print(df_first_10)
    
#Detect on the large amounts from transfer 300
    
    


# Conclusion